In [1]:
import os
%pwd

'/home/dhanush/Documents/MLOPS/SummarizerText_Hf_End2End_1/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/dhanush/Documents/MLOPS/SummarizerText_Hf_End2End_1'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config


In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

/home/dhanush/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-20 17:55:59,463: INFO: config: PyTorch version 2.5.1 available.]


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading the data
        dataset_dialogsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_dialogsum_pt["test"],
                  eval_dataset=dataset_dialogsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [9]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: accelerate 1.1.1
Uninstalling accelerate-1.1.1:
  Successfully uninstalled accelerate-1.1.1
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Using cached accelerate-1.1.1-py3-none-any.whl (333 kB)


In [10]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2024-11-20 17:57:03,620: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-20 17:57:03,624: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-20 17:57:03,626: INFO: common: created directory at: artifacts]
[2024-11-20 17:57:03,628: INFO: common: created directory at: artifacts/model_trainer]


OSError: Incorrect path_or_model_id: 'artifacts/model_trainer/checkpoint'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [14]:
from box import ConfigBox

# Assuming your configuration is loaded from a file or defined here
config = ConfigBox({
    'model_ckpt': 'path_to_checkpoint_file'
})

# Safe access using get() method
model_ckpt = config.get('model_ckpt', None)
if model_ckpt:
    print(f"Model checkpoint path: {model_ckpt}")
else:
    print("The 'model_ckpt' key does not exist.")



Model checkpoint path: path_to_checkpoint_file


In [12]:
if 'model_ckpt' in config:
    model_ckpt = config['model_ckpt']
else:
    print("model_ckpt key not found!")


In [14]:
print(config)

In [16]:
model_ckpt = config.get('model_ckpt', None)  # Default to None if the key doesn't exist
if model_ckpt:
    print(f"Checkpoint: {model_ckpt}")
else:
    print("The key 'model_ckpt' does not exist.")


Checkpoint: path_to_checkpoint_file


NameError: name 'self' is not defined